In [1]:
%cd /home/thenodev/Projects/GutenbergOnomasticsExplorer

/home/thenodev/Projects/GutenbergOnomasticsExplorer


In [2]:
import langchain

In [ ]:
import pandas as pd

In [ ]:
from langchain.document_loaders import GutenbergLoader

In [ ]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

In [ ]:
import chromadb

In [15]:
df = pd.read_parquet("./data/metadata.parquet")

In [16]:
df = df.groupby("title").agg(lambda x: x.tolist()[0] if len(set(x.tolist()))== 1 else x.tolist()).reset_index()

In [77]:
df_sample = df.sample(10000)

In [78]:
def getURL(list_):
    try:
        txt_url = list(filter(lambda x: x.endswith(".txt"), list_))
    except:
        txt_url = ""
    return txt_url

In [79]:
df_sample['download_url'] = df_sample.hasFormat.apply(getURL)

In [80]:
df_sample['download_url'] = df_sample.download_url.apply(lambda x: x[0] if len(x)>= 1 else "")

In [81]:
samples = df_sample.where(df_sample.download_url != "").sample(100)

In [82]:
def getText(q):
    try:
        loader = GutenbergLoader(q)
        doc = loader.load()
        return doc
    except:
        return

In [83]:
samples['docs'] = samples.download_url.apply(lambda x: getText(x))

In [97]:
samples['text'] = samples.docs.apply(lambda x: x[0].page_content if x is not None else "")

In [100]:
small_ds = samples[samples.text != ""]

In [102]:
small_ds.to_json("./data/small_dataset_gutenberg.json")

In [103]:
def chunkText(q):
    splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = splitter.split_documents(q)
    return docs


In [104]:
small_ds['chunks'] = small_ds.docs.apply(chunkText)

/tmp/ipykernel_1426/1139083163.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_ds['chunks'] = small_ds.docs.apply(chunkText)


In [111]:
small_ds = small_ds[small_ds.language=="en"]

In [115]:
from langchain.embeddings import CohereEmbeddings

In [118]:
embedding_function = CohereEmbeddings(cohere_api_key="2m72a8NYuZmXQLdfwsH6LQ1SzZbgN6jPZVW5CQdh")

In [120]:
small_ds = small_ds.explode("chunks")

In [122]:
small_ds['chunks'] = small_ds.chunks.apply(lambda x: x.page_content)

In [126]:
embeds = embedding_function.aembed_documents(small_ds.chunks)

In [133]:
import time

In [134]:
embeds = []
for i, row in small_ds.iterrows():
    embeds.append(embedding_function.embed_query(row.chunks))
    time.sleep(1)

KeyboardInterrupt: 

In [136]:
small_ds

,title,hasFormat,publisher,license,issued,rights,downloads,creator_name,creator_birthdate,creator_deathdate,...,subject3,subject4,bookshelf1,bookshelf2,bookshelf3,download_url,docs,texxt,text,chunks
9413,"Chambers's Journal of Popular Literature, Scie...",[https://www.gutenberg.org/ebooks/46959.html.i...,Project Gutenberg,http://www.gutenberg.org/license,2014-09-25,Public domain in the USA.,24,Various,None,None,...,None,None,Chambers's Edinburgh Journal,None,None,https://www.gutenberg.org/files/46959/46959-0.txt,[page_content='The Project Gutenberg EBook of ...,The Project Gutenberg EBook of Chambers's Jour...,The Project Gutenberg EBook of Chambers's Jour...,The Project Gutenberg EBook of Chambers's Jour...
9413,"Chambers's Journal of Popular Literature, Scie...",[https://www.gutenberg.org/ebooks/46959.html.i...,Project Gutenberg,http://www.gutenberg.org/license,2014-09-25,Public domain in the USA.,24,Various,None,None,...,None,None,Chambers's Edinburgh Journal,None,None,https://www.gutenberg.org/files/46959/46959-0.txt,[page_content='The Project Gutenberg EBook of ...,The Project Gutenberg EBook of Chambers's Jour...,The Project Gutenberg EBook of Chambers's Jour...,Produced by Susan Skinner and the Online Distr...
9413,"Chambers's Journal of Popular Literature, Scie...",[https://www.gutenberg.org/ebooks/46959.html.i...,Project Gutenberg,http://www.gutenberg.org/license,2014-09-25,Public domain in the USA.,24,Various,None,None,...,None,None,Chambers's Edinburgh Journal,None,None,https://www.gutenberg.org/files/46959/46959-0.txt,[page_content='The Project Gutenberg EBook of ...,The Project Gutenberg EBook of Chambers's Jour...,The Project Gutenberg EBook of Chambers's Jour...,along with boys and girls belonging to the tow...
9413,"Chambers's Journal of Popular Literature, Scie...",[https://www.gutenberg.org/ebooks/46959.html.i...,Project Gutenberg,http://www.gutenberg.org/license,2014-09-25,Public domain in the USA.,24,Various,None,None,...,None,None,Chambers's Edinburgh Journal,None,None,https://www.gutenberg.org/files/46959/46959-0.txt,[page_content='The Project Gutenberg EBook of ...,The Project Gutenberg EBook of Chambers's Jour...,The Project Gutenberg EBook of Chambers's Jour...,"known as Galashiels blue, with two broad metal..."
9413,"Chambers's Journal of Popular Literature, Scie...",[https://www.gutenberg.org/ebooks/46959.html.i...,Project Gutenberg,http://www.gutenberg.org/license,2014-09-25,Public domain in the USA.,24,Various,None,None,...,None,None,Chambers's Edinburgh Journal,None,None,https://www.gutenberg.org/files/46959/46959-0.txt,[page_content='The Project Gutenberg EBook of ...,The Project Gutenberg EBook of Chambers's Jour...,The Project Gutenberg EBook of Chambers's Jour...,"at school, he took the shine out of boys decke..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,"A Complete History of Music\r\nfor Schools, Cl...",[https://www.gutenberg.org/ebooks/54392.html.i...,Project Gutenberg,http://www.gutenberg.org/license,2017-03-20,Public domain in the USA.,241,"Baltzell, W. J. (Winton James)",1864,1928,...,None,None,None,None,None,https://www.gutenberg.org/files/54392/54392-0.txt,[page_content='Project Gutenberg\'s A Complete...,Project Gutenberg's A Complete History of Musi...,Project Gutenberg's A Complete History of Musi...,generations. To learn more about the Project G...
805,"A Complete History of Music\r\nfor Schools, Cl...",[https://www.gutenberg.org/ebooks/54392.html.i...,Project Gutenberg,http://www.gutenberg.org/license,2017-03-20,Public domain in the USA.,241,"Baltzell, W. J. (Winton James)",1864,1928,...,None,None,None,None,None,https://www.gutenberg.org/files/54392/54392-0.txt,[page_content='Project Gutenberg\'s A Complete...,Project Gutenberg's A Complete History of Musi...,Project Gutenberg's A Complete History of Musi...,locations. Its business office is located at 8...
805,"A Complete History of Music\r\nfor Schools, Cl...",[https://www.guten